In [2]:
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import kachery as ka
import os
from utils import prepare_dataset_from_hash
import numpy as np
import spikesorters as ss

os.environ["TMP"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["TMPDIR"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["TEMPDIR"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["ML_TEMPORARY_DIRECTORY"] = "/home/mclancy/SpikeConfidence/.tmp"

os.environ["KACHERY_STORAGE_DIR"] = "/data/.kache"

# Configure kachery to download data from the public database
ka.set_config(fr='default_readonly')

base_dir = Path('/home/mclancy/SpikeConfidence/')
cache_path = Path('/data/.cache')
spike_sorter_dir = base_dir / 'spikesorters'

# kilosort2_5
ss.Kilosort2_5Sorter.set_kilosort2_5_path(spike_sorter_dir / 'Kilosort')

ss.IronClustSorter.set_ironclust_path(spike_sorter_dir / 'ironclust')

/home/mclancy/.local/share/pyenv/versions/SpikeConfidence/lib/python3.8/site-packages/kachery/_update_config_repos.py:39: ResourceWarning: unclosed file <_io.BufferedReader name=62>
  _pull_git_repo(path=config_repo_path + '/repo')
/home/mclancy/.local/share/pyenv/versions/SpikeConfidence/lib/python3.8/site-packages/kachery/_update_config_repos.py:39: ResourceWarning: unclosed file <_io.BufferedReader name=64>
  _pull_git_repo(path=config_repo_path + '/repo')


Setting KILOSORT2_5_PATH environment variable for subprocess calls to: /home/mclancy/SpikeConfidence/spikesorters/Kilosort
Setting IRONCLUST_PATH environment variable for subprocess calls to: /home/mclancy/SpikeConfidence/spikesorters/ironclust


In [3]:
static_recording_path = 'sha1dir://615aa23efde8898aa89002613e20ad59dcde42f9.hybrid_janelia/static_siprobe/rec_16c_1200s_11'
static_gt_path = 'sha1dir://615aa23efde8898aa89002613e20ad59dcde42f9.hybrid_janelia/static_siprobe/rec_16c_1200s_11/firings_true.mda'

drift_recording_path = 'sha1dir://615aa23efde8898aa89002613e20ad59dcde42f9.hybrid_janelia/drift_siprobe/rec_16c_1200s_11'
drift_gt_path = 'sha1dir://615aa23efde8898aa89002613e20ad59dcde42f9.hybrid_janelia/drift_siprobe/rec_16c_1200s_11/firings_true.mda'

sorter_names = ['mountainsort4', 'herdingspikes', 'spykingcircus', 'ironclust','tridesclous']

metric_names = np.array(["num_spikes", "firing_rate", "presence_ratio",
                "isi_violation", "amplitude_cutoff", "snr",
                "max_drift", "cumulative_drift", "silhouette_score",
                "isolation_distance", "l_ratio",
                "nn_hit_rate", "nn_miss_rate","d_prime"])

/home/mclancy/.local/share/pyenv/versions/SpikeConfidence/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
X_static, y_static = prepare_dataset_from_hash(recording_path=static_recording_path, gt_path=static_gt_path, metric_names=metric_names, sorter_names=sorter_names, cache_path=cache_path)
X_drift, y_drift = prepare_dataset_from_hash(recording_path=drift_recording_path, gt_path=drift_gt_path, metric_names=metric_names, sorter_names=sorter_names, cache_path=cache_path)

In [5]:
model = make_pipeline(StandardScaler(), LogisticRegression())

X_train, X_test, y_train, y_test = train_test_split(X_drift, y_drift, test_size=0.33, random_state=0)

scaler = StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

# False positive classification via logistic regression
model.fit(X_train_transformed, y_train)
model_accuracy = model.score(X_test_transformed, y_test)
print(f"Full metrics model accuracy is {model_accuracy}")

Full metrics model accuracy is 0.875
